# Stitching together patterns

Sometimes you got different patterns and you wanna stitch them together.

Like, say I want a kick drum pattern that's like `f([3/4, 1 1/2, 2])` for 3 cycles, then one cycle of `f([1/2, 1], offset=1/4)`.

It's often useful to stitch stuff together like this. So, how could we do this?

I guess it makes sense to specify the regions of each function you wanna stitch together, in which case you might as well just specify the length of each piece you're stitching.

Here's what I mean: you can take the first pattern I mentioned and specify that you want to take 6 beats of that, giving:

In [ ]:
from harmonica.time import OnsetFunc
from harmonica.utility._mixed import Mixed


pattern_selection_a = OnsetFunc([Mixed(x) for x in ["3/4", "1 1/2", 2]]).to_clip(
    Mixed(6)
)

Then you take 2 beats of another pattern:

In [ ]:
pattern_selection_b = OnsetFunc(
    [Mixed(x) for x in [1, 2]], offset=Mixed("1/2")
).to_clip(Mixed(2))

And you stitch em together somehow.

To be clear, I want to splice the onset functions together into a new onset function, not splice clips together... although splicing clips IS something I wanna do at another point!

We need to write a function that does this.

I guess it'd just take a list of tuples. Each tuple will consist of a onset function, and how many beats to use from that function.

Time for some scratch work...

## Scratch work

I worked it out in my notebook, and I think the easiest strategy is

1. Have a variable called duration initialized at 0, and a list called onsets initialized empty
2. Get a set of onsets from an `in_range` call, with a start value of 0 and a stop value of our splice duration (from the tuple)
3. Add our duration variable to each of the onsets we get from this `in_range` call
4. Append these onsets to our list of onsets, and add the aforementioned duration from the tuple to our duration variable
5. Once we're done iterating through the tuples, store the first onset in our onsets list as the offset value
6. Subtract this offset value from each of the onsets in our list, pop off the first element (which should now be 0), and add the duration value to the end
7. This new modified list is our final onset function pattern, and together with the offset value we stored, we now have the resulting onset function!

In [ ]:
def basic_timefunc_splicer(splices: list[tuple[OnsetFunc, Mixed]]) -> OnsetFunc:
    duration = 0
    onsets = []

    for onseofunc, splice_dur in splices:
        of_onsets = onseofunc.in_range(Mixed(0), Mixed(splice_dur))
        of_onsets = [onset + duration for onset in of_onsets]
        onsets.extend(of_onsets)
        duration += splice_dur

    offset = onsets[0]
    pattern = [onset - offset for onset in onsets]
    pattern = pattern[1:] + [duration]

    return OnsetFunc(pattern, offset)

Looks good to me! Let's test the darn thing.

In [4]:
from harmonica.time._event._clip import Clip
from harmonica.utility._gm import GMDrum


pat_a = OnsetFunc([Mixed(x) for x in ["3/4", "1 1/2", 2]], offset=Mixed("1/4"))
pat_b = OnsetFunc([Mixed(x) for x in [1, 2]], offset=Mixed("1/2"))

spliced_pattern = basic_timefunc_splicer([(pat_a, Mixed(6)), (pat_b, Mixed(2))])
clave = spliced_pattern.to_clip(Mixed(32))
kicks = OnsetFunc([Mixed(1)]).to_clip(Mixed(32), drum=GMDrum.AcousticBassDrum)

# Clip([clave, kicks]).preview()
# print(Clip([clave, kicks]))

Damn... that's awesome!

### How the hell do I represent silence, though?

Rests are a big part of music, because silence is a big part of music. There has to be some way of representing rests, at least in certain contexts.

Take my time func splicer, for example. What if I want to make a time func that represents a rhythm where I just want to insert big gaps?

Like, maybe I want to have a pattern go on for 5 beats, and then have a pause for 3 beats, then another pattern for 8 beats.

The first idea that came to mind was having empty onset functions to represent silence. But this seems like an abuse of the concept of a onset function.

The next idea is to just have a simple Rest object. An event that just consists of an onset and a duration. 

But how would this fit into the onset function splicer? It takes a list of tuples with an object and a duration. It'd make no sense to have to specify splice duration in the tuple when the rest object already specifies a duration.

Oh, wait. I can just type the parameter differently. Instead of `list[tuple[OnsetFunc, Mixed]]`, type it as `list[tuple[OnsetFunc, Mixed] | Rest]`.

That seems like it'd work. Let's try...

In [ ]:
from harmonica.time import OnsetFunc, Mixed, Rest


def timefunc_splicer(splices: list[tuple[OnsetFunc, Mixed] | Rest]) -> OnsetFunc:
    duration = 0
    onsets = []

    for splice in splices:
        if isinstance(splice, tuple):
            onseofunc, splice_dur = splice
            of_onsets = onseofunc.in_range(Mixed(0), Mixed(splice_dur))
            of_onsets = [onset + duration for onset in of_onsets]
            onsets.extend(of_onsets)
            duration += splice_dur
        if isinstance(splice, Rest):
            duration += splice.duration

    offset = onsets[0]
    pattern = [onset - offset for onset in onsets]
    pattern = pattern[1:] + [duration]

    return OnsetFunc(pattern, offset)

That simple, eh? Let's test it:

In [6]:
pat_a = OnsetFunc([Mixed(x) for x in ["3/4", "1 1/2", 2]], offset=Mixed("1/4"))
pat_b = OnsetFunc([Mixed(x) for x in [1, 2]], offset=Mixed("1/2"))

spliced_pattern = basic_timefunc_splicer([(pat_a, Mixed(6)), (pat_b, Mixed(2))])
clave = spliced_pattern.to_clip(Mixed(128))
kicks = OnsetFunc([Mixed(1)]).to_clip(Mixed(128), drum=GMDrum.AcousticBassDrum)

pat_c = timefunc_splicer(
    [Rest(Mixed(4)), (OnsetFunc([Mixed(x) for x in ["1/4", "3/4"]]), Mixed(4))]
)
pat_c = timefunc_splicer(
    [
        (pat_c, Mixed(28)),
        (
            OnsetFunc(
                [Mixed(x) for x in ["1/3", "2/3", "1 1/3", "1 2/3", 2, 4]],
                offset=Mixed("1/3"),
            ),
            Mixed(4),
        ),
    ]
)
hats = pat_c.to_clip(Mixed(128), drum=GMDrum.ClosedHiHat)

# Clip([kicks, hats, clave]).preview()

Really neat!

The unfortunate thing is, this syntax is hideous and tedious. But, baby steps...

## Some OnsetFunc methods

I wish I could be like, 

```
pat_a = ...
pat_b = ...
pat_c = ...
pat_d = pat_a.trunc(Mixed(8)).concat(pat_b.trunc(Mixed(4))).concat(pat_c.trunc(Mixed(4)))
```

I made the truncate method for OnsetFunc. Let's see...

In [7]:
f = OnsetFunc([Mixed(x) for x in ["3/4", "1 1/2", 2]], Mixed("1 1/2"))
g = f.trunc(Mixed("5 1/2"))
dur = Mixed(128)

# Clip(
#     [
#         f.to_clip(dur, drum=GMDrum.Claves),
#         g.to_clip(dur, drum=GMDrum.Cowbell),
#     ]
# ).preview()

Works! Okay, now I need a concatenate method.

In [8]:
f = OnsetFunc(pattern=[Mixed(x) for x in ["3/4", "1 1/2", 2]], offset=Mixed("1 1/2"))
g = OnsetFunc(pattern=[Mixed(x) for x in [1, 2, 3, 4]])
h = f.trunc(Mixed("6 1/2")).concat(g.trunc(Mixed(3)))

# Clip(
#     [
#         h.trunc(Mixed(16)).to_clip(duration=Mixed(128)),
#         OnsetFunc([Mixed(1)]).to_clip(Mixed(128), drum=GMDrum.AcousticBassDrum),
#     ]
# ).preview()

Whoa! Yeah, this is really intuitive! I like this a lot.

I really like having truncate and concatenate be their own methods. First of all, using method chaining is intuitive (same as it ever was), and:
- Truncate is useful for when you have a complicated pattern and want it to repeat after a certain number of beats. This is great for imposing regularity to irregular rhythms.
- Concatenate is useful for building compound rhythms.

### But seriously, what about silence?

What if I have a rhythmic pattern that I want to start with 4 beats of rest, and then go into some other pattern for 6 beats?

`Rest(4).concat(f)`, right? This feels kind of awkward, because I have to introduce this method to Rest that specifically returns a OnsetFunc object. Feels weird for some reason.

Another idea is to have an empty OnsetFunc. But then you can't specify the modulus, which is needed to specify how long the rest is.

What if there's a method that lets you insert silence into a pattern? Multiple ideas:

- pad_tail(x) adds x beats of silence to the end, pad_head(x) adds x beats of silence to the start
- rest(onset, x) inserts x beats of silence at time `onset`

rest would give more control, but it'd be annoying to insert silence at the end. I mean, for real, what would I type? `f.rest(f.modulus, 4)`?

meanwhile, `f.pad_tail(4)` is more succinct, and `f.pad_head(4)` is pretty neat too. but `f.rest(0,4)` actually does read pretty well.

Something like `f.rest(-1, 4)` would be a neat way of representing "add 4 beats of rest to the end".

Eh. Maybe just add them all? Hahahhah!

Nah, I think I'll just add pad and pad_tail. pad_head doesn't seem necessary, because it's already plenty easy to just write a 0 at the start.

In [ ]:
f = OnsetFunc([Mixed(x) for x in ["3/4", "1 1/2", 2]]).trunc(Mixed(5))

f = f.pad_tail(Mixed(f.modulus))

f.to_clip(Mixed(128)).preview()